In [1]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Gesell

This example demonstrates ...

Adapted from [unit 9: Gesell.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/Gesell.odc).



Note: not really working right now. Will go over it again later.



Associated lecture video: Unit 9 Lesson 6

## Problem statement

Gesell Developmental Schedules  measure  child development. The scoring is devised by the American child psychologist and pediatrician Arnold Gesell who founded the Clinic of Child Development at Yale in 1911 and directed it for many years.  The Gesell Developmental Schedules are a gauge of the status of a child's motor and language development and personal-social and adaptive behaviors.

For 21 children the age in months at which they first spoke and their Gesell Adaptive Score, which is the result of an aptitude test taken much later was recorded.  These data were originally collected by L. M. Linde of UCLA but were first published by M. R. Mickey, O. J. Dunn, and V. Clark, †Note on the use of stepwise regression in detecting outliers,† Computers and Biomedical Research, 1 (1967), pp. 105†111. 

The data have been used by several authors, e.g.,  N. R. Draper and
J. A. John, †Influential observations and outliers in regression,† 
Technometrics, 23 (1981), pp. 21†26.

Let X be the age in in months a child speaks his/her first word
and let Y be the Gesell adaptive score, a measure of a child†s
aptitude (observed later on). 


Correlation of r = ¢0.640 indicates a  moderately strong negative
relationship between age at first word spoken and Gesell score.
 How does the child†s aptitude change with how long it takes them to speak?

Are there any influential observations/potential outliers?

The solution follows a two stage modeling. 

(A) Obtain mean ICPOs. Outside BUGS take reciprocal of mean ICPOs.
(these are CPOs) and consder them as an input for model B.

(B) Read in and calibrate CPOs with posterior ordinates


In [3]:
# fmt: off
x = np.array(
    (15, 26, 10, 9, 15, 20, 18, 11, 8, 20, 7, 9, 10, 11, 11, 10, 12, 42, 17, 11, 10)
)

y = np.array(
    (95, 71, 83, 91, 102, 87, 93, 100, 104, 94, 113, 96, 83, 84, 102, 100, 105, 57, 121, 86, 100)
)
# fmt: on

## Model A

In [4]:
with pm.Model() as model:
    # couldn't get .001 working, answers are pretty close with .003
    tau = pm.Gamma("tau", 1, 0.003)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.0001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "cpo", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    # what is the point of this when we want cpo anyways?
    # icpo = pm.Deterministic("icpo", 1 / f)
    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    trace = pm.sample(10000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, b]


Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 201 seconds.


In [5]:
cpo_df = az.summary(trace, var_names="cpo", round_to=5, filter_vars="like")

In [6]:
cpo_means = cpo_df["mean"].to_numpy()
cpo_means

array([0.03596, 0.02394, 0.01318, 0.02634, 0.02533, 0.03624, 0.03462,
       0.03539, 0.03442, 0.02983, 0.02104, 0.03431, 0.01318, 0.01689,
       0.03314, 0.03604, 0.02599, 0.02643, 0.00107, 0.02081, 0.03604])

## Model B


In [7]:
mlcpo = -2 * np.log(cpo_means)
MPQ = np.sum(mlcpo)

In [8]:
with pm.Model() as model:
    # couldn't get .001 working, answers are pretty close with .003
    tau = pm.Gamma("tau", 1, 0.003)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.0001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "f", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    c = pm.Deterministic("c", cpo_means / f)
    # clean this up later
    g1 = pm.Deterministic("g1", c * pm.math.log(c))
    g2 = -pm.math.log(c)
    g3 = 0.5 * pm.math.abs_(c - 1)
    g4 = (c - 1) ** 2

    pm.Deterministic("s_g1", pm.math.sum(g1))
    pm.Deterministic("s_g2", pm.math.sum(g2))
    pm.Deterministic("s_g3", pm.math.sum(g3))
    pm.Deterministic("s_g4", pm.math.sum(g4))

    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    traceb = pm.sample(2000, target_accept=0.95)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, b]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 36 seconds.


In [9]:
az.summary(traceb, var_names="c", filter_vars="like")

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
c[0],1.026,0.172,0.721,1.336,0.003,0.002,3539.0,4444.0,1.0
c[1],1.155,0.599,0.554,2.107,0.009,0.006,4906.0,5768.0,1.0
c[2],1.204,0.655,0.478,2.259,0.010,0.007,3932.0,4141.0,1.0
c[3],1.059,0.278,0.647,1.555,0.005,0.003,3808.0,5150.0,1.0
c[4],1.040,0.223,0.699,1.466,0.003,0.002,6131.0,5699.0,1.0
c[5],1.027,0.177,0.731,1.361,0.003,0.002,3245.0,4163.0,1.0
c[6],1.027,0.178,0.729,1.364,0.003,0.002,3830.0,4518.0,1.0
c[7],1.027,0.177,0.724,1.355,0.003,0.002,3465.0,4405.0,1.0
c[8],1.031,0.196,0.698,1.382,0.004,0.003,3265.0,3415.0,1.0
c[9],1.042,0.228,0.682,1.454,0.003,0.002,5179.0,5458.0,1.0


In [10]:
az.waic(traceb, scale="deviance")

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/stats.py:1661: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 8000 posterior samples and 21 observations log-likelihood matrix.

              Estimate       SE
deviance_waic   165.39    10.10
p_waic            3.41        -

There has been a warning during the calculation. Please check the results.

In [11]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

numpy: 1.22.4
pymc : 4.0.0
arviz: 0.12.1

